## Import Libraries

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time

## Download dataset 

`transform.compose` allows you to perform more than one transform on the dataset.

`transform.ToTensor` converts your data to tensors

`transform.Normalize` normalize your data with the mean and standard deviation, this makes the data comparable to each other 


`trainset` is the training dataset

`valset` is the validation dataset 

the corresponding loaders is used to load the data into the neural network later on, the batch size tells you how many data to load at a time


In [ ]:
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training data
trainset = datasets.MNIST('drive/My Drive/mnist/MNIST_data/', download=True, train=True, transform=transform)
valset = datasets.MNIST('drive/My Drive/mnist/MNIST_data/', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

Below we make the trainloader an iterative type

Re-run the cell a few times and see more different examples

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
plt.imshow(images[0].numpy().squeeze(), cmap='gray_r');

In [ ]:
figure = plt.figure()
num_of_images = 60
for index in range(1, num_of_images + 1):
    plt.subplot(6, 10, index)
    plt.axis('off')
    plt.imshow(images[index].numpy().squeeze(), cmap='gray_r')

Look at our data and their types.
What does the shape values represent?

In [ ]:
print(type(images))
print(images.shape)
print(labels.shape)

## Create a Convolutional Neural Network 

Look at the example here:
https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

The architecture to follow for the neural network we are building:

ConvNet(
>>  (layer1):
>>> Sequential(

>>> (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))

>>> (1): ReLU()

>>> (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

>> )

>>  (layer2): 
>>> Sequential(

>>> (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))

>>> (1): ReLU()

>>>( 2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

>>)

>>> (drop_out): Dropout(p=0.5)
  
>>> (fc1): Linear(in_features=3136, out_features=1000, bias=True)

>>> (fc2): Linear(in_features=1000, out_features=10, bias=True)
)

In [ ]:
# Hyperparameters
num_epochs = 6
num_classes = 10
learning_rate = 0.001

from torch import nn
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
model = ConvNet()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Print the model below and check if you created the model with the right architecture

In [ ]:
print(model)

## Train your model

In [ ]:
total_step = len(trainloader)
# Keep track of the loss
loss_list = []
# Keep track of the accuracy
acc_list = []

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

## Testing and Evaluation

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model: {} %'.format((correct / total) * 100))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def view_classify(img, ps, version="MNIST"):
    ''' Function for viewing an image and it's predicted classes.
    '''
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    if version == "MNIST":
        ax2.set_yticklabels(np.arange(10))
    elif version == "Fashion":
        ax2.set_yticklabels(['T-shirt/top',
                            'Trouser',
                            'Pullover',
                            'Dress',
                            'Coat',
                            'Sandal',
                            'Shirt',
                            'Sneaker',
                            'Bag',
                            'Ankle Boot'], size='small');
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

In [ ]:
images, labels = next(iter(valloader))
with torch.no_grad():
    # get the prediction by passing in an image to the model
    o=model(images)
    # Take exponent of this so value is between 0 and 1
    ps = torch.exp(o)
    ps= ps[0][:]
    probab = list(ps.numpy())
    # The maximum value in the list is the predicted label of the image
    print("Predicted Digit =", probab.index(max(probab)))
    # To get the probability take this number and divide by the total sum
    b=torch.sum(ps)
    p=ps/b
    # visualize your data
    view_classify(images[0].view(1, 28, 28), p)

## Testing on Air-written Digits

Read an image in

use `cv2.resize(img, (28,28), interpolation = cv2.INTER_AREA)` to resize your image to match the MNIST data

use `cv2.cvtColor` to convert the image to grayscale

apply previously defined `transform ` to normalize the image

set the image to be viewed the same way by the neural network by calling `view(1,1,28,28)` on the image

Use your model to predict the label of your image, use `view_classify` to show your image and the predicted probabilities


In [ ]:
pass